#### Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  

The Enron dataset contains data for 146 Enron employees with 21 features including one feature called POI (person of interest) to mark employees who conducted fraudulent activites. The features can be divided into 3 categories:  
    
   - 14 financial features that relate to employees' financial charateristics, e.g. salary, bonus and total stock value etc.;
   - 6 email features that counting number of emails created from/to a person and from/to a POI, e.g. number of emails from POIs to this person, number of emails from this person to POIs etc..
   - 1 label feature that is the POI column to indicate whether the employee is an POI or not. There are 18 employees that have been labelled as POIs and 128 employees labelled as non-POIs.
   
Except email address field and the label field, all other features are numeric.

The goal of this project is to use a variaty of machine learning algorithms to classify employees to POI or non-POI category based on financial features and email features. The classification result of each algorithm will then be validated against the label feature and we will use different metrics to measure and compare the quality of each classification model. 

An outlier with name 'TOTAL' was identified and removed while plotting the salary - bonus scatter plot, as this is the aggregation row for all employee data in the dataset. Also by checking the data manually, a row named 'THE TRAVEL AGENCY IN THE PARK' was also removed as this not seems like a name of a person and therefore removed.  
   
After removing outliner, there are 18 POIs and 126 non-POIs.

While doing data wrangling and EDA (referring to Wrangling and EDA.ipynb), I found that in both finacial features and email features there are NaN values. Considering the size of the dataset is small (144 data points after outlier removal), I am not going to remove these data points. By looking at the histogram of each feature, I found that for most of the features, the distribution is heavily skewed. Therefore NaNs are replaced with median value of the feature. 



#### What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  

Email address is removed from the feature list as the text of email address will not help in classifying employees.   
Intuitively I would think the proportion of emails to or from a POI to the total number of messages of a person may tell something. In other words, the more a person has interactions with POIs, the higher the chance that this person is also a POI. So I have created a column called 'poi_messages_total_messages_ratio'. This column is showing the result of 
<br>
```
    (from_poi_to_this_person + from_this_person_to_poi) / (to_messages + from_messages) 
```
<br>

A boxplot (see details in Wrangling and EDA.ipynb) shows that in general POIs have higher poi_messages_total_messages_ratio than non-POIs meaning that comparing to non-POIs, POIs have more email interactions with other POIs. So I kept this created feature in my features list for automated feature selection used in the implementation of pipeline of different algorithms. 

StandardScaler, SelectKBest and PCA algorithms together with KNeighborsClassifier were implemented in a Pipeline object. We need to scale the features because we use both finance features and email features to create the classification model. The finance features are having bigger variance than email features. If scaling is not adopted, most likely PCA will ignore email features even though email features still have great impact on determine if an employee is POI or not. 

By Using GridSearchCV I was able to give the pipeline a range of numbers and let the GridSearchCV help to find out the best number of features and the best features. GridSearchCV found when k = 13 - meaning that it automatically found the best 13 features and fed these 13 features into the KNeighborsClassifier, it can give the best f1 score.
These 13 features and their associated feature scores and p-values are listed below. We can see that the newly created feature 'poi_messages_total_messages_ratio' had been selected to be one of the best features.

|Feature Name | Feature Score | Feature P-Value |
| --- | --- | --- |
|exercised_stock_options | 27.45 | 0.000 |
|total_stock_value | 23.67 | 0.000 |
|bonus | 15.80 | 0.000 |
|salary | 10.90 | 0.001 |
|deferred_income | 10.29 | 0.002 |
|restricted_stock | 8.46 | 0.004 |
|total_payments | 8.41 | 0.004 |
|long_term_incentive | 8.36 | 0.004 |
|shared_receipt_with_poi | 7.48 | 0.007 |
|from_poi_to_this_person | 4.28 | 0.040 |
|other | 3.96 | 0.049 |
|poi_messages_total_messages_ratio | 3.87 | 0.051 |
|loan_advances | 3.85 | 0.052 |

ExtraTreesClassifier was another used machine learning algorithm. No automated feature selection was adopted since the algorithm would calculate the feature importace and create a split on the feathre that is separating class labels the best. 

Here is the feature importance calculated by ExtraTreesClassifier. It shows that the added feature 'poi_messages_total_messages_ratio' actually has the highest importance among all other features.

|Feature Rank | Feature Name | Feature Importance |
| --- | --- | --- |
|1 | poi_messages_total_messages_ratio | 0.152698924925 |
|2 | total_payments | 0.12825665974 |
|3 | total_stock_value | 0.127183785477 |
|4 | long_term_incentive | 0.10263509473 |
|5 | exercised_stock_options | 0.0844120640939 |
|6 | deferred_income | 0.0731560200504 |
|7 | restricted_stock | 0.0729123903489 |
|8 | bonus | 0.0588591716643 |
|9 | expenses | 0.0402617974295 |
|10 | from_poi_to_this_person | 0.0374883652549 |
|11 | salary | 0.0291464081023 |
|12 | to_messages | 0.0268306710977 |
|13 | other | 0.0202748518273 |
|14 | deferral_payments | 0.0138725747541 |
|15 | from_messages | 0.0101448131742 |
|16 | director_fees | 0.00998833683408 |
|17 | shared_receipt_with_poi | 0.00620816501592 |
|18 | from_this_person_to_poi | 0.00329180032199 |
|19 | restricted_stock_deferred | 0.00237810515773 |
|20 | loan_advances | 0.0 |

#### What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?

I used 2 algorithms to be the classification models - ExtraTreesClassifier and KNeighborsClassifier.

KNeighborsClassifier algorithm gave better performance in terms of the accuracy of the classification based on Precision and Recall score. KNeighboursClassifer also performed faster with the give parameter grid than ExtraTreesClassifier with the given parameter grid.

Here is the comparison of the test report of the two algorithms based on 100-fold StratifiedShuffleSplit cross-validation when training the data and then 1000-fold StratifiedShuffleSplit cross-validation:

|KNeighborsClassifier|
| --- |
|<b>Metric</b> | <b>Score</b> |
|Accuracy | 0.87080 |
|Precision | 0.52120 |
|Recall | 0.38100 |
|F1 | 0.44021 |
|F2 | 0.40266 |
|Total predictions | 15000 |
|True positives |  762 |
|False positives |  700 |
|False negatives | 1238 |
|True negatives | 12300 |
|Time Used | 1892 Seconds |


|ExtraTreesClassifier|
| --- |
|<b>Metric</b> | <b>Score</b> |
|Accuracy |  0.80167 |
|Precision |  0.30979 |
|Recall |  0.39700 |
|F1 |  0.34802 |
|F2 |  0.37584 |
|Total predictions |  15000 |
|True positives | 794 |
|False positives | 1769 |
|False negatives | 1206 |
|True negatives | 11231 |
|Time Used | 9019 Seconds |

#### What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? What parameters did you tune? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric items: “discuss parameter tuning”, “tune the algorithm”]

Tuning parameter of the machine learning algorithm is the process of optimising the parameters to specify the algorithm to achieve the maximized the predictive power. 

The best parameters of a selected algorithm would always highly depend on the dataset itself, therefore for each machine learning problem, as long as we are using the different dataset, we will always need to tune the parameters, otherwise the algorithm could barely give an ideal result.

I had used GridSearchCV for parameter tuning for both ExtraTreeClassifier and KNeighborsClassifier. 

For ExtraTreesClassifer, I made the GridSearchCV to try on the below combinations of paramaters:

|Parameter Name | Options |
| --- | --- |
|n_estimators | [10, 20, 30, 40, 50] |
|criterion | ['gini', 'entropy'] |
|max_features | ['sqrt'] |
|max_depth | [None] |
|min_samples_split | [2, 3, 4] |
|min_samples_leaf | [1, 2, 3] |
|min_weight_fraction_leaf | [0.0] |
|max_leaf_nodes | [None] |
|bootstrap | [False] |
|oob_score | [False] |
|n_jobs | [-1] |
|random_state | [42] |
|class_weight | ['balanced', 'balanced_subsample'] |

For KNeighborsClassifier, I tried paramaters not only on the classifier, but also on the scaler, feature selection and PCA algorithms:

| scaler__with_std | [True, False] |
| --- | --- |
| skb__k | range(5, 15)
| pca__n_components | range(1,5)
| clf__weights | ['distance', 'uniform'] |
| clf__algorithm | ['auto', 'ball_tree', 'kd_tree', 'brute'] |
| clf__n_neighbors | [3,5,10] |

These parameter options were chose with the default values as reference, together with some trials and experiments. For ExtraTreesClassifier, the focus was on the number of trees in the forest, the tree specification like the minimum number of samples required to split an internal node, and the minimum number of samples required to be at a leaf node etc. 
Considering the two algorithms together, the set of parmeters that gives the best f1 score is:

In [24]:
print
[
        {'StandardScaler': {'copy': True,
                            'with_mean': True,
                            'with_std': False
                            }
        },
        {'SelectKBest': {'k': 13,
                         'score_func': 'f_classif'
                        }
        },
        {'PCA': {'copy': True,
                 'iterated_power': 'auto',
                 'n_components': 1,
                 'random_state': 42,
                 'svd_solver': 'auto',
                 'tol': 0.0,
                 'whiten': False
                }
        },
        {'KNeighborsClassifier': {'algorithm': 'auto',
                                     'leaf_size': 30,
                                     'metric': 'minkowski',
                                     'metric_params': None,
                                     'n_jobs': 1,
                                     'n_neighbors': 5,
                                     'p': 2,
                                     'weights': 'distance'
                                 }
        }
]


[{'StandardScaler': {'copy': True, 'with_mean': True, 'with_std': False}},
 {'SelectKBest': {'k': 13, 'score_func': 'f_classif'}},
 {'PCA': {'copy': True,
   'iterated_power': 'auto',
   'n_components': 1,
   'random_state': 42,
   'svd_solver': 'auto',
   'tol': 0.0,
   'whiten': False}},
 {'KNeighborsClassifier': {'algorithm': 'auto',
   'leaf_size': 30,
   'metric': 'minkowski',
   'metric_params': None,
   'n_jobs': 1,
   'n_neighbors': 5,
   'p': 2,
   'weights': 'distance'}}]

And the best test result by the best parameter tested by test_classifier from tester.py with 1000-fold StratifiedShuffleSplit is:

|Metric | Score |
| --- | --- |
|Accuracy | 0.87080 |
|Precision | 0.52120 |
|Recall | 0.38100 |
|F1 | 0.44021 |
|F2 | 0.40266 |
|Total predictions | 15000 |
|True positives |  762 |
|False positives |  700 |
|False negatives | 1238 |
|True negatives | 12300 |
|Time Used | 1892 Seconds |

#### What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric items: “discuss validation”, “validation strategy”]

Validation is the process of testing the model's performance (model is trained from a training set) with the tuned parameters in terms of specified metrics. Validation is usually done by testing the trained model on a test dataset.

A classic mistake if validation is done wrong would be overfitting, which means that the model has been tuned to perfectly predict the labels from the training set, however when predicting unknow data points, the model is not reliable in generalise the characteristics of the data.

To avoid overfitting, I used StratifiedShuffleSplit for cross validation when training my model. Because the Enron data set is unbalanced (Number of POIs are much less than number of non-POIs), StratifiedShuffleSplit ensures that the randomised folds are made by preserving the percentage of samples for each class.


#### Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]

Precision and Recall are two common metrics to evaluate the performance of a classification model. Looking at the average performance of the KNeighborsClassifier:

|Metric | Score |
| --- | --- |
|Accuracy | 0.87080 |
|Precision | 0.52120 |
|Recall | 0.38100 |
|F1 | 0.44021 |
|F2 | 0.40266 |
|Total predictions | 15000 |
|True positives |  762 |
|False positives |  700 |
|False negatives | 1238 |
|True negatives | 12300 |

Precision means that for all the employees that had been identified as POI, 52.12% are true POIs.
Recall means that for all true POIs in the dataset, 38.1% are identified by the model as true POIs.
F1 is a weighted metric of Precision and Recall. The higher F1 usually means the higher Precision and Recall.